In [66]:
semestri = []

for i in range(16,25):
    leto = "FRI " + "20" + str(i) + "/20" + str(i + 1)
    semestri.append(leto + ", zimski semester")
    semestri.append(leto + ", letni semester")
semestri


['FRI 2016/2017, zimski semester',
 'FRI 2016/2017, letni semester',
 'FRI 2017/2018, zimski semester',
 'FRI 2017/2018, letni semester',
 'FRI 2018/2019, zimski semester',
 'FRI 2018/2019, letni semester',
 'FRI 2019/2020, zimski semester',
 'FRI 2019/2020, letni semester',
 'FRI 2020/2021, zimski semester',
 'FRI 2020/2021, letni semester',
 'FRI 2021/2022, zimski semester',
 'FRI 2021/2022, letni semester',
 'FRI 2022/2023, zimski semester',
 'FRI 2022/2023, letni semester',
 'FRI 2023/2024, zimski semester',
 'FRI 2023/2024, letni semester',
 'FRI 2024/2025, zimski semester',
 'FRI 2024/2025, letni semester']

In [67]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time


BASE_URL = "https://urnik.fri.uni-lj.si/timetable" 
all_data = []

driver = webdriver.Chrome()
driver.get(BASE_URL)


In [68]:

try:

    p_element = driver.find_element(By.TAG_NAME, "p")
    links = p_element.find_elements(By.TAG_NAME, "a")




except Exception as e:
    print(f"Napaka pri semestru: {e}")

seznam = [(link.text, link.get_attribute("href")) for link in links]
seznam.reverse()

prve_pojavitve = {}

for text, href in seznam:
    for sem in semestri:
        if sem in text and sem not in prve_pojavitve:
            prve_pojavitve[sem] = (text, href)


rezultat = list(prve_pojavitve.values())
rezultat


[('FRI 2024/2025, zimski semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2024_2025-zimski'),
 ('FRI 2023/2024, zimski semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2023_2024-zimski'),
 ('FRI 2023/2024, letni semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2023_2024-letni'),
 ('FRI 2024/2025, letni semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2024_2025-letni'),
 ('FRI 2020/2021, zimski semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2020_2021-zimski'),
 ('FRI 2021/2022, zimski semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2021_2022-zimski'),
 ('FRI 2022/2023, zimski semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2022_2023-zimski'),
 ('FRI 2022/2023, letni semester-1-10',
  'https://urnik.fri.uni-lj.si/timetable/fri-2022_2023-letni-1-10'),
 ('FRI 2021/2022, letni semester',
  'https://urnik.fri.uni-lj.si/timetable/fri-2021_2022-letni-1-1'),
 ('FRI 2020/2021, letni semester-1-9',
  'https://urnik.fri.uni-lj.si/timetable/fri-2

In [69]:
povezava  = "https://urnik.fri.uni-lj.si/timetable/fri-2024_2025-zimski"

driver.get(povezava)
try:
    link_element = driver.find_element(By.XPATH, "//a[contains(text(), '1. letnik, Računalništvo in informatika, prva stopnja: visokošolski strokovni')]")
    link_element.click()
except Exception as e:
    print(f"Napaka pri kliku na element: {e}")

predmeti = []

try:
    elements_with_p = [element for element in driver.find_elements(By.XPATH, "//a[contains(text(), '_P')]") if element.text.endswith('_P')]
    for element in elements_with_p:
        text = element.text
        href = element.get_attribute("href")
        predmeti.append((text, href))
except Exception as e:
    print(f"Napaka pri iskanju elementov z '_P': {e}")

predmeti


[('DS(63705)_P',
  'https://urnik.fri.uni-lj.si/timetable/fri-2024_2025-zimski/allocations?subject=63705'),
 ('RA_P',
  'https://urnik.fri.uni-lj.si/timetable/fri-2024_2025-zimski/allocations?subject=63703'),
 ('M_P',
  'https://urnik.fri.uni-lj.si/timetable/fri-2024_2025-zimski/allocations?subject=63704'),
 ('P1_P',
  'https://urnik.fri.uni-lj.si/timetable/fri-2024_2025-zimski/allocations?subject=63702'),
 ('UR_P',
  'https://urnik.fri.uni-lj.si/timetable/fri-2024_2025-zimski/allocations?subject=63701')]

In [70]:
kratica_ime = {}

kratica, povezava = predmeti[0]

driver.get(povezava)

try:
    subtitle_element = driver.find_element(By.CLASS_NAME, "subtitle")
    kratica_ime[kratica] = subtitle_element.text
except Exception as e:
    print(f"Napaka pri pridobivanju elementa z razredom 'subtitle': {e}")


print(kratica_ime)

{'DS(63705)_P': 'Diskretne strukture'}
